In [1]:
import glob
import copy
import numpy as np
import scipy
import astropy.table
import astropy.io.fits
import astropy.units

import matplotlib.pyplot as plt
import matplotlib as mpl


import lezargus

import pycpd

%matplotlib Qt

ToDoError - Uncertainty values on integrations need to be done.
ToDoError - Uncertainty values on integrations need to be done.
ToDoError - Uncertainty values on integrations need to be done.
ToDoError - Uncertainty values on integrations need to be done.
ToDoError - Uncertainty values on integrations need to be done.
ToDoError - Uncertainty values on integrations need to be done.
ToDoError - Uncertainty values on integrations need to be done.
ToDoError - Uncertainty values on integrations need to be done.
ToDoError - Uncertainty values on integrations need to be done.
ToDoError - Uncertainty values on integrations need to be done.
ToDoError - Uncertainty values on integrations need to be done.
ToDoError - Uncertainty values on integrations need to be done.


# Slice Pattern File

In [2]:
# Reading the base file, and saving the new file.
slice_pattern_filename = "./base/spectre_slice_pattern.txt"
pattern_data = np.genfromtxt(slice_pattern_filename, comments="#").T

pattern_slice_index = pattern_data[0]
pattern_slice_x = pattern_data[1]
pattern_slice_y = pattern_data[2]

# The pattern values in the file are given in millimeters, we convert it to
# meters per convention.
pattern_slice_x = pattern_slice_x / 1000
pattern_slice_y = pattern_slice_y / 1000

pattern_index_column = astropy.table.Column(
    pattern_slice_index, name="index", unit="", dtype=int
)
pattern_x_column = astropy.table.Column(
    pattern_slice_x, name="x", unit="m", dtype=float
)
pattern_y_column = astropy.table.Column(
    pattern_slice_y, name="y", unit="m", dtype=float
)

table_columns = [pattern_index_column, pattern_x_column, pattern_y_column]
pattern_table = astropy.table.Table(table_columns)
# Too much precision makes unnessary large file sizes.
pattern_filename = f"./products/spectre_slice_pattern.dat"
pattern_table.write(
    pattern_filename,
    format="ascii.mrt",
    formats={"index": "%4d"}
    | {keydex: "%.3e" for keydex in pattern_table.keys()},
    overwrite=True,
)
pattern_pairs = np.array([pattern_slice_x, pattern_slice_y]).T

# Disperstion Table File

In [3]:
# Reading the template file which we will use to generate all of the
# other points.
template_dispersion_filename = "./base/spectre_slice_dispersion.txt"
template_dispersion_table = astropy.table.Table.read(
    template_dispersion_filename,
    format="ascii.basic",
    delimiter="|",
    comment="#",
)
# Convert to meters.
template_dispersion_table["Left X"] /= 1000
template_dispersion_table["Left Y"] /= 1000
template_dispersion_table["Center X"] /= 1000
template_dispersion_table["Center Y"] /= 1000
template_dispersion_table["Right X"] /= 1000
template_dispersion_table["Right Y"] /= 1000

In [18]:
def get_points_dispersion_table(
    channel: str, wavelength: float, location: str
) -> np.ndarray:
    """Get the points from the dispersion table.

    Parameters
    ----------
    channel : str
        The channel: VIS, NIR, MIR.
    wavelength : float
        The wavelength desired, but it must exist.
    location : str
        The location of the points within the slice, either "Left", "Center",
        or "Right".

    Returns
    -------
    points : ndarray
        The points.
    """
    # We filter the main table.
    filter_channel = template_dispersion_table["Channel"] == channel
    filter_wavelength = template_dispersion_table["Wavelength"] == wavelength
    filter_index = filter_channel & filter_wavelength
    filtered_table = template_dispersion_table[filter_index]

    point_x = np.array(filtered_table[f"{location} X"])
    point_y = np.array(filtered_table[f"{location} Y"])
    points = np.array([point_x, point_y]).T
    return points


def transform_pattern(pattern: np.ndarray, anchors: np.ndarray) -> np.ndarray:
    """Register/transform the provided point pattern to fit the anchor points.

    Parameters
    ----------
    pattern : ndarray
        The point pattern that we are registering. For most cases, this is the
        36 slice point pattern.
    anchors : ndarray
        The anchor points we are fitting the pattern to. For most cases, this
        is the wavelength dispersion slice points.

    Returns
    -------
    new_pattern : ndarray
        The point pattern after being transformed to fit the anchor points.
    """
    # The order of the pattern and anchors is important.
    transformation_mode = "affine"
    if transformation_mode == "rigid":
        registration = pycpd.RigidRegistration(X=anchors, Y=pattern)
        new_pattern, (_scale, _rotation, _translation) = registration.register()
    elif transformation_mode == "affine":
        registration = pycpd.AffineRegistration(X=anchors, Y=pattern)
        new_pattern, (_matrix, _translation) = registration.register()
    else:
        raise ValueError("Unknown transformation mode.")
    # As the registration above does the transformations for us, we can ignore
    # them.

    print(_matrix)
    print(_translation)
    return new_pattern


def channel_long_name(short_name: str):
    """Just, write out the channel name.

    Parameters
    ----------
    short_name : str
        The short name: VIS, NIR, MIR

    Returns
    -------
    long_name : str
        The long name.
    """
    channel_translation = {"VIS": "visible", "NIR": "nearir", "MIR": "midir"}
    return channel_translation[short_name]

In [16]:
# The unique combinations avaliable for the dispersion file.
dispersion_combinations = (
    # Channel, wavelength
    ["VIS", 0.40],
    ["VIS", 0.50],
    ["VIS", 0.60],
    ["VIS", 0.70],
    ["VIS", 0.80],
    ["VIS", 0.85],
    # ...
    ["NIR", 0.85],
    ["NIR", 1.25],
    ["NIR", 2.40],
    # ...
    ["MIR", 2.40],
    ["MIR", 3.20],
    ["MIR", 4.00],
    ["MIR", 4.20],
)
# The anchor points do not provide any indication of height, so we calcuate
# the height of the slice assuming that the height to width ratio is always the
# same. Height / Width.
height_width_ratio = 2 / 72


# Color map functionality for debugging plots.
def color_function(wavelength: float):
    # Parameters
    min_wave = 0.40
    max_wave = 4.20

    # We use energy as map.
    min_energy = 1 / max_wave
    max_energy = 1 / min_wave
    energy = 1 / wavelength

    color_map = copy.deepcopy(mpl.colormaps["jet_r"])
    normalize = mpl.colors.Normalize(
        vmin=min_energy, vmax=max_energy, clip=True
    )
    color = color_map(normalize(energy))
    return color

### Doing the work...

In [19]:
# Table rows.
full_dispersion_rows = []

plt.figure()
for channeldex, wavedex in dispersion_combinations:
    # Left
    left_anchor = get_points_dispersion_table(
        channel=channeldex, wavelength=wavedex, location="Left"
    )
    left_pattern = transform_pattern(pattern=pattern_pairs, anchors=left_anchor)
    # Center
    center_anchor = get_points_dispersion_table(
        channel=channeldex, wavelength=wavedex, location="Center"
    )
    center_pattern = transform_pattern(
        pattern=pattern_pairs, anchors=center_anchor
    )
    # Right
    right_anchor = get_points_dispersion_table(
        channel=channeldex, wavelength=wavedex, location="Right"
    )
    right_pattern = transform_pattern(
        pattern=pattern_pairs, anchors=right_anchor
    )

    # We use the width to figure out the top and bottom points based on the
    # ratio.
    left_x_points = left_pattern.T[0]
    right_x_points = right_pattern.T[0]
    widths = np.abs(right_x_points - left_x_points)
    heights = height_width_ratio * widths
    half_height = heights / 2

    # We assume the top and bottom points are directly offset in Y by the half
    # height from the center.
    top_pattern = np.array(
        [
            [xdex, ydex + heightdex]
            for (xdex, ydex), heightdex in zip(center_pattern, half_height)
        ]
    )
    bot_pattern = np.array(
        [
            [xdex, ydex - heightdex]
            for (xdex, ydex), heightdex in zip(center_pattern, half_height)
        ]
    )

    # Debug plot.
    if True:
        # The found pattern points, in reverse order of importance.
        plt.scatter(*bot_pattern.T, marker=".", color=color_function(wavedex))
        plt.scatter(*top_pattern.T, marker=".", color=color_function(wavedex))
        plt.scatter(*right_pattern.T, marker=".", color=color_function(wavedex))
        plt.scatter(*left_pattern.T, marker=".", color=color_function(wavedex))
        plt.scatter(
            *center_pattern.T, marker=".", color=color_function(wavedex)
        )
        # And the anchors to compare it to.
        plt.scatter(*left_anchor.T, marker="2", color="pink")
        plt.scatter(*center_anchor.T, marker="*", color="pink")
        plt.scatter(*right_anchor.T, marker="2", color="pink")
        # Millimeter coordinate tick marks are easier.
        plt.show()

    for sliceindex, (cdex, tdex, bdex, ldex, rdex) in enumerate(
        zip(
            center_pattern,
            top_pattern,
            bot_pattern,
            left_pattern,
            right_pattern,
        )
    ):
        # Convert wavelength to meters.
        wavedex_m = wavedex * 1e-6

        row_dict = {
            "channel": channel_long_name(short_name=channeldex),
            "slice": sliceindex + 1,
            "wavelength": wavedex_m * astropy.units.Unit("m"),
            "center_x": cdex[0] * astropy.units.Unit("m"),
            "center_y": cdex[1] * astropy.units.Unit("m"),
            "top_x": tdex[0] * astropy.units.Unit("m"),
            "top_y": tdex[1] * astropy.units.Unit("m"),
            "bottom_x": bdex[0] * astropy.units.Unit("m"),
            "bottom_y": bdex[1] * astropy.units.Unit("m"),
            "left_x": ldex[0] * astropy.units.Unit("m"),
            "left_y": ldex[1] * astropy.units.Unit("m"),
            "right_x": rdex[0] * astropy.units.Unit("m"),
            "right_y": rdex[1] * astropy.units.Unit("m"),
        }
        full_dispersion_rows.append(row_dict)

plt.show()

[[8.33266201e-01 1.69009897e-04]
 [8.96605414e-04 8.20192014e-01]]
[-0.0005235   0.00050406]
[[8.33248562e-01 1.79026013e-04]
 [8.02564902e-04 8.20119001e-01]]
[-8.02764244e-08  5.04166388e-04]
[[ 8.32832192e-01  1.78791185e-04]
 [-2.07753919e-04  8.20210491e-01]]
[0.00052256 0.00050409]
[[ 8.33929781e-01 -6.75164623e-05]
 [ 9.14944197e-04  8.20779648e-01]]
[-0.00052164 -0.00282206]
[[8.34018485e-01 1.76576374e-04]
 [8.06139747e-04 8.20670017e-01]]
[ 2.85917692e-06 -2.82154725e-03]
[[8.33974620e-01 4.22079003e-04]
 [6.90389737e-04 8.21462419e-01]]
[ 0.00052471 -0.00282078]
[[ 8.34312427e-01 -2.11657304e-04]
 [ 3.20502075e-04  8.21091767e-01]]
[-0.00051448 -0.00463071]
[[8.36182184e-01 1.84784020e-04]
 [8.58364290e-04 8.21055747e-01]]
[ 2.24772512e-05 -4.63017936e-03]
[[8.34342112e-01 5.23018785e-04]
 [6.90465295e-04 8.21074243e-01]]
[ 0.00052541 -0.0046309 ]
[[ 8.34552188e-01 -3.05035520e-04]
 [ 8.59963221e-04  8.21275419e-01]]
[-0.00051934 -0.00577377]
[[8.34524003e-01 1.54280273e-04]

In [15]:
# Creating and saving the table.
full_dispersion_table = astropy.table.Table(rows=full_dispersion_rows)

# Types
table_type_formats = {
    "channel": str,
    "slice": "{:d}",
    "wavelength": "{: .3e}",
    "center_x": "{: .9f}",
    "center_y": "{: .9f}",
    "top_x": "{: .9f}",
    "top_y": "{: .9f}",
    "bottom_x": "{: .9f}",
    "bottom_y": "{: .9f}",
    "left_x": "{: .9f}",
    "left_y": "{: .9f}",
    "right_x": "{: .9f}",
    "right_y": "{: .9f}",
}

full_dispersion_filename = f"./products/spectre_slice_dispersion.dat"
full_dispersion_table.write(
    full_dispersion_filename,
    format="ascii.mrt",
    formats=table_type_formats,
    overwrite=True,
)

In [16]:
test_dispersion_table = astropy.table.Table.read(
    full_dispersion_filename,
    format="ascii.mrt",
)